In [0]:
import pandas as pd
import os 
import requests

In [0]:
# Import alpha_vantage_key.txt and save it as API_KEY
with open("alpha_vantage_key.txt", "r") as file:
    API_KEY = file.read().strip()

In [0]:
# Base URL for Alpha Vantage
BASE_URL = "https://www.alphavantage.co/query"

# We’ll use the TIME_SERIES_INTRADAY function for hourly data
params = {
    "function": "TIME_SERIES_INTRADAY",
    "symbol": "SPY",           # S&P 500 ETF proxy
    "interval": "60min",       # could also be 1min, 5min, 15min, 30min, 60min
    "outputsize": "compact",   # 'compact' returns the last 100 data points, 'full' returns entire available history
    "apikey": API_KEY
}

response = requests.get(BASE_URL, params=params)
data = response.json()

# data will contain a dict with metadata and time series. 
# The keys typically look like:
# "Meta Data" and "Time Series (60min)" (depending on the interval).
# Example of how to parse out the time series:
time_series_key = "Time Series (60min)"
if time_series_key in data:
    ts_data = data[time_series_key]
    for timestamp, values in ts_data.items():
        open_price = values["1. open"]
        high_price = values["2. high"]
        low_price  = values["3. low"]
        close_price = values["4. close"]
        volume = values["5. volume"]
        print(f"{timestamp} | O:{open_price} H:{high_price} L:{low_price} C:{close_price} V:{volume}")
else:
    print("Error: No time series data found. Check the response or parameters.")


In [0]:
# Build a list of dicts
records = []
for timestamp, metrics in data['Time Series (60min)'].items():
    record = {
        "timestamp": timestamp,
        "open": float(metrics["1. open"]),
        "high": float(metrics["2. high"]),
        "low": float(metrics["3. low"]),
        "close": float(metrics["4. close"]),
        "volume": int(metrics["5. volume"])
    }
    records.append(record)

# Create a DataFrame
df = pd.DataFrame(records)
# Optionally, convert timestamp to a Pandas datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

df.head()

In [0]:
df.shape

In [0]:
data['Meta Data']
data['Time Series (60min)']